In [1]:
#Sensitive Encryption key
cryptokey='AIxyLjsFocf-NEuEXRS_VUJ1XEEkf4nE9QQ2cWnhlro='

Installing dependencies

In [ ]:
!pip3 install amodem uuid cryptography

Importing necessary libraries

In [2]:
import os
import uuid
from ipywidgets import FileUpload
import base64
from cryptography.fernet import Fernet

Setting bitrate for 2kbps

In [3]:
def setBitrate():
    os.environ['BITRATE']='16'

In [4]:
setBitrate()

 Calibration

Check audio output device

In [ ]:
!amodem send --calibrate

Interrupt the kernel when calibration is done

In [5]:
#Variable for filename
global fln
fln=''

Handle upload

In [6]:
def on_upload_change(change):
    global fln
    if not change.new:
        return
    up = change.owner
    for filename,data in up.value.items():
        with open(filename, 'wb') as f:
            fln=filename
            f.write(data['content'])
    up.value.clear()
    up._counter = 0

upload_btn = FileUpload()
upload_btn.observe(on_upload_change, names='_counter')
upload_btn


FileUpload(value={}, description='Upload')

In [7]:
print("Filename: "+fln)

Filename: demo.mp4


In [8]:
file = open(fln, 'rb')
file_content = file.read()
file.close()

Generate metadata: Filename and its length

In [9]:
flnlen=len(fln).to_bytes(1,byteorder='big')
metadata=flnlen+fln.encode()
data=metadata+file_content

In [ ]:
data

Encrypt

In [10]:
fkey = Fernet(cryptokey.encode())
encdata=fkey.encrypt(data)
tempdata=encdata

In [ ]:
encdata

Combine metadata and data and write to a temporary file

In [11]:
tempname=str(uuid.uuid4())
with open(tempname, 'wb') as f:
    f.write(tempdata)
os.environ['tempname']=tempname

In [ ]:
tempname

In [12]:
tempsize = os.path.getsize(tempname)
print(f"Attempting to send {tempsize} bytes")
print("Make sure receiver is listening")

Attempting to send 1680228 bytes
Make sure receiver is listening


Sending

In [ ]:
!echo $BITRATE

In [ ]:
!amodem send -vv -i $tempname

Cleanup: Delete temporary file and uploaded file

In [13]:
os.remove(fln)
os.remove(tempname)